In [16]:
# force geopandas to use shapely instead of pygeos
import os
os.environ['USE_PYGEOS'] = '0'

import numpy as np
import pandas as pd
import plotly.express as px
from shapely import wkt
from shapely.geometry import Point
import geopandas as gpd
import matplotlib.pyplot as plt
import seaborn as sns
import datetime

In [17]:
pd.set_option('display.max_columns', 500)

# Load in .csv of Final Probabilities for All Streets

In [18]:
final_probs = pd.read_csv("../data/final_probs.csv")

In [19]:
final_probs.head()

,streets,days,09:00:00,09:15:00,09:30:00,09:45:00,10:00:00,10:15:00,10:30:00,10:45:00,11:00:00,11:15:00,11:30:00,11:45:00,12:00:00,12:15:00,12:30:00,12:45:00,13:00:00,13:15:00,13:30:00,13:45:00,14:00:00,14:15:00,14:30:00,14:45:00,15:00:00,15:15:00,15:30:00,15:45:00,16:00:00,16:15:00,16:30:00,16:45:00,17:00:00,17:15:00,17:30:00,17:45:00
0,Mission St 07th St - Julia St,Monday,0.037037,0.037037,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.037037,0.0,0.0,0.074074,0.000000,0.037037,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
1,Mission St 07th St - Julia St,Tuesday,0.037037,0.037037,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.037037,0.037037,0.037037,0.037037,0.000000,0.000000,0.037037,0.037037,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
2,Mission St 07th St - Julia St,Wednesday,0.000000,0.037037,0.000000,0.0,0.037037,0.0,0.037037,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.037037,0.000000,0.037037,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
3,Mission St 07th St - Julia St,Thursday,0.074074,0.037037,0.074074,0.0,0.000000,0.0,0.000000,0.037037,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.037037,0.000000,0.037037,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.037037
4,Mission St 07th St - Julia St,Friday,0.074074,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.037037,0.037037,0.000000,0.000000,0.000000,0.0,0.0,0.037037,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000


In [20]:
# if any street has probability of 0 for every day of the week, remove it

street_group = final_probs.groupby("streets").agg("sum").sum(axis=1)
exclude_list = street_group[street_group == 0].index
final_probs = final_probs[~final_probs['streets'].isin(exclude_list)]

/var/folders/b7/y91htkqs5jn2g1znl4ghlk1r0000gn/T/ipykernel_2885/3996723233.py:3: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  street_group = final_probs.groupby("streets").agg("sum").sum(axis=1)


In [21]:
final_probs.shape

(12264, 38)

In [22]:
# only select the columns of times

int_cols = [col for col in final_probs.columns if col not in ["days", "streets"]]
df_new = final_probs[int_cols]


In [23]:
# create new columns of 0's to account for 2 hour parking after 11:45pm

a = pd.Series(0, index=df_new.index, name='A')
b = pd.Series(0, index=df_new.index, name='B')
c = pd.Series(0, index=df_new.index, name='C')
d = pd.Series(0, index=df_new.index, name='D')
e = pd.Series(0, index=df_new.index, name='E')
f = pd.Series(0, index=df_new.index, name='F')
g = pd.Series(0, index=df_new.index, name='G')
h = pd.Series(0, index=df_new.index, name='H')

df_new = pd.concat([df_new, a,b,c,d,e,f,g,h], axis=1)

In [24]:
df_new

,09:00:00,09:15:00,09:30:00,09:45:00,10:00:00,10:15:00,10:30:00,10:45:00,11:00:00,11:15:00,11:30:00,11:45:00,12:00:00,12:15:00,12:30:00,12:45:00,13:00:00,13:15:00,13:30:00,13:45:00,14:00:00,14:15:00,14:30:00,14:45:00,15:00:00,15:15:00,15:30:00,15:45:00,16:00:00,16:15:00,16:30:00,16:45:00,17:00:00,17:15:00,17:30:00,17:45:00,A,B,C,D,E,F,G,H
0,0.037037,0.037037,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.037037,0.0,0.000000,0.074074,0.000000,0.037037,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0,0,0,0,0,0,0,0
1,0.037037,0.037037,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.037037,0.037037,0.037037,0.037037,0.000000,0.000000,0.037037,0.037037,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0,0,0,0,0,0,0,0
2,0.000000,0.037037,0.000000,0.0,0.037037,0.0,0.037037,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.037037,0.000000,0.037037,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0,0,0,0,0,0,0,0
3,0.074074,0.037037,0.074074,0.0,0.000000,0.0,0.000000,0.037037,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.037037,0.000000,0.037037,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.037037,0,0,0,0,0,0,0,0
4,0.074074,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.037037,0.037037,0.000000,0.000000,0.000000,0.0,0.0,0.037037,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14772,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0,0,0,0,0,0,0,0
14773,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0,0,0,0,0,0,0,0
14774,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0,0,0,0,0,0,0,0
14775,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.030303,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0,0,0,0,0,0,0,0


In [25]:
# create new columns for 7 offsets per time, each offset moving by 15 mins
# probabilities will be probability of NOT getting a parking ticket in that interval calculated by prod_j=1^k(1-p_j)

multiplied_values_dict = dict()

# loop over time columns in the dataframe (the -8 is so we stop after 11:45 since we now have A-H columns at the end)
for start_col in range(df_new.shape[1]-8):
    running_product = 1-df_new.iloc[:, start_col]

   # creates key for dictionary based on offset number and multiplies probabilities
    for i in range(start_col+1, start_col+9):
        new_key = f"{df_new.columns[start_col]}_{i-start_col-1}"
        multiplied_values_dict[new_key] = (1-df_new.iloc[:, i]) * running_product
        running_product *= (1-df_new.iloc[:, i])


In [26]:
# want probability of getting a ticket, so substract probabilities from 1

offset_df = 1 - pd.DataFrame(multiplied_values_dict)
offset_df

,09:00:00_0,09:00:00_1,09:00:00_2,09:00:00_3,09:00:00_4,09:00:00_5,09:00:00_6,09:00:00_7,09:15:00_0,09:15:00_1,09:15:00_2,09:15:00_3,09:15:00_4,09:15:00_5,09:15:00_6,09:15:00_7,09:30:00_0,09:30:00_1,09:30:00_2,09:30:00_3,09:30:00_4,09:30:00_5,09:30:00_6,09:30:00_7,09:45:00_0,09:45:00_1,09:45:00_2,09:45:00_3,09:45:00_4,09:45:00_5,09:45:00_6,09:45:00_7,10:00:00_0,10:00:00_1,10:00:00_2,10:00:00_3,10:00:00_4,10:00:00_5,10:00:00_6,10:00:00_7,10:15:00_0,10:15:00_1,10:15:00_2,10:15:00_3,10:15:00_4,10:15:00_5,10:15:00_6,10:15:00_7,10:30:00_0,10:30:00_1,10:30:00_2,10:30:00_3,10:30:00_4,10:30:00_5,10:30:00_6,10:30:00_7,10:45:00_0,10:45:00_1,10:45:00_2,10:45:00_3,10:45:00_4,10:45:00_5,10:45:00_6,10:45:00_7,11:00:00_0,11:00:00_1,11:00:00_2,11:00:00_3,11:00:00_4,11:00:00_5,11:00:00_6,11:00:00_7,11:15:00_0,11:15:00_1,11:15:00_2,11:15:00_3,11:15:00_4,11:15:00_5,11:15:00_6,11:15:00_7,11:30:00_0,11:30:00_1,11:30:00_2,11:30:00_3,11:30:00_4,11:30:00_5,11:30:00_6,11:30:00_7,11:45:00_0,11:45:00_1,11:45:00_2,11:45:00_3,11:45:00_4,11:45:00_5,11:45:00_6,11:45:00_7,12:00:00_0,12:00:00_1,12:00:00_2,12:00:00_3,12:00:00_4,12:00:00_5,12:00:00_6,12:00:00_7,12:15:00_0,12:15:00_1,12:15:00_2,12:15:00_3,12:15:00_4,12:15:00_5,12:15:00_6,12:15:00_7,12:30:00_0,12:30:00_1,12:30:00_2,12:30:00_3,12:30:00_4,12:30:00_5,12:30:00_6,12:30:00_7,12:45:00_0,12:45:00_1,12:45:00_2,12:45:00_3,12:45:00_4,12:45:00_5,12:45:00_6,12:45:00_7,13:00:00_0,13:00:00_1,13:00:00_2,13:00:00_3,13:00:00_4,13:00:00_5,13:00:00_6,13:00:00_7,13:15:00_0,13:15:00_1,13:15:00_2,13:15:00_3,13:15:00_4,13:15:00_5,13:15:00_6,13:15:00_7,13:30:00_0,13:30:00_1,13:30:00_2,13:30:00_3,13:30:00_4,13:30:00_5,13:30:00_6,13:30:00_7,13:45:00_0,13:45:00_1,13:45:00_2,13:45:00_3,13:45:00_4,13:45:00_5,13:45:00_6,13:45:00_7,14:00:00_0,14:00:00_1,14:00:00_2,14:00:00_3,14:00:00_4,14:00:00_5,14:00:00_6,14:00:00_7,14:15:00_0,14:15:00_1,14:15:00_2,14:15:00_3,14:15:00_4,14:15:00_5,14:15:00_6,14:15:00_7,14:30:00_0,14:30:00_1,14:30:00_2,14:30:00_3,14:30:00_4,14:30:00_5,14:30:00_6,14:30:00_7,14:45:00_0,14:45:00_1,14:45:00_2,14:45:00_3,14:45:00_4,14:45:00_5,14:45:00_6,14:45:00_7,15:00:00_0,15:00:00_1,15:00:00_2,15:00:00_3,15:00:00_4,15:00:00_5,15:00:00_6,15:00:00_7,15:15:00_0,15:15:00_1,15:15:00_2,15:15:00_3,15:15:00_4,15:15:00_5,15:15:00_6,15:15:00_7,15:30:00_0,15:30:00_1,15:30:00_2,15:30:00_3,15:30:00_4,15:30:00_5,15:30:00_6,15:30:00_7,15:45:00_0,15:45:00_1,15:45:00_2,15:45:00_3,15:45:00_4,15:45:00_5,15:45:00_6,15:45:00_7,16:00:00_0,16:00:00_1,16:00:00_2,16:00:00_3,16:00:00_4,16:00:00_5,16:00:00_6,16:00:00_7,16:15:00_0,16:15:00_1,16:15:00_2,16:15:00_3,16:15:00_4,16:15:00_5,16:15:00_6,16:15:00_7,16:30:00_0,16:30:00_1,16:30:00_2,16:30:00_3,16:30:00_4,16:30:00_5,16:30:00_6,16:30:00_7,16:45:00_0,16:45:00_1,16:45:00_2,16:45:00_3,16:45:00_4,16:45:00_5,16:45:00_6,16:45:00_7,17:00:00_0,17:00:00_1,17:00:00_2,17:00:00_3,17:00:00_4,17:00:00_5,17:00:00_6,17:00:00_7,17:15:00_0,17:15:00_1,17:15:00_2,17:15:00_3,17:15:00_4,17:15:00_5,17:15:00_6,17:15:00_7,17:30:00_0,17:30:00_1,17:30:00_2,17:30:00_3,17:30:00_4,17:30:00_5,17:30:00_6,17:30:00_7,17:45:00_0,17:45:00_1,17:45:00_2,17:45:00_3,17:45:00_4,17:45:00_5,17:45:00_6,17:45:00_7
0,0.072702,0.072702,0.072702,0.072702,0.072702,0.072702,0.072702,0.072702,0.037037,0.037037,0.037037,0.037037,0.037037,0.037037,0.037037,0.072702,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.037037,0.037037,0.000000,0.000000,0.000000,0.000000,0.000000,0.037037,0.037037,0.037037,0.000000,0.000000,0.000000,0.000000,0.037037,0.037037,0.037037,0.108368,0.000000,0.000000,0.000000,0.037037,0.037037,0.037037,0.108368,0.108368,0.000000,0.000000,0.037037,0.037037,0.037037,0.108368,0.108368,0.141391,0.000000,0.037037,0.037037,0.037037,0.108368,0.108368,0.141391,0.141391,0.037037,0.037037,0.037037,0.108368,0.108368,0.141391,0.141391,0.141391,0.037037,0.037037,0.108368,0.108368,0.141391,0.141391,0.141391,0.141391,0.000000,0.074074,0.074074,0.108368,0.108368,0.108368,0.108368,0.108368,0.074074,0.074074,0.108368,0.108368,0

In [12]:
# adding back street name and weekday columns

offset_with_streets = pd.concat([final_probs[["days", 'streets']], offset_df], axis=1)
offset_with_streets

,days,streets,09:00:00_0,09:00:00_1,09:00:00_2,09:00:00_3,09:00:00_4,09:00:00_5,09:00:00_6,09:00:00_7,09:15:00_0,09:15:00_1,09:15:00_2,09:15:00_3,09:15:00_4,09:15:00_5,09:15:00_6,09:15:00_7,09:30:00_0,09:30:00_1,09:30:00_2,09:30:00_3,09:30:00_4,09:30:00_5,09:30:00_6,09:30:00_7,09:45:00_0,09:45:00_1,09:45:00_2,09:45:00_3,09:45:00_4,09:45:00_5,09:45:00_6,09:45:00_7,10:00:00_0,10:00:00_1,10:00:00_2,10:00:00_3,10:00:00_4,10:00:00_5,10:00:00_6,10:00:00_7,10:15:00_0,10:15:00_1,10:15:00_2,10:15:00_3,10:15:00_4,10:15:00_5,10:15:00_6,10:15:00_7,10:30:00_0,10:30:00_1,10:30:00_2,10:30:00_3,10:30:00_4,10:30:00_5,10:30:00_6,10:30:00_7,10:45:00_0,10:45:00_1,10:45:00_2,10:45:00_3,10:45:00_4,10:45:00_5,10:45:00_6,10:45:00_7,11:00:00_0,11:00:00_1,11:00:00_2,11:00:00_3,11:00:00_4,11:00:00_5,11:00:00_6,11:00:00_7,11:15:00_0,11:15:00_1,11:15:00_2,11:15:00_3,11:15:00_4,11:15:00_5,11:15:00_6,11:15:00_7,11:30:00_0,11:30:00_1,11:30:00_2,11:30:00_3,11:30:00_4,11:30:00_5,11:30:00_6,11:30:00_7,11:45:00_0,11:45:00_1,11:45:00_2,11:45:00_3,11:45:00_4,11:45:00_5,11:45:00_6,11:45:00_7,12:00:00_0,12:00:00_1,12:00:00_2,12:00:00_3,12:00:00_4,12:00:00_5,12:00:00_6,12:00:00_7,12:15:00_0,12:15:00_1,12:15:00_2,12:15:00_3,12:15:00_4,12:15:00_5,12:15:00_6,12:15:00_7,12:30:00_0,12:30:00_1,12:30:00_2,12:30:00_3,12:30:00_4,12:30:00_5,12:30:00_6,12:30:00_7,12:45:00_0,12:45:00_1,12:45:00_2,12:45:00_3,12:45:00_4,12:45:00_5,12:45:00_6,12:45:00_7,13:00:00_0,13:00:00_1,13:00:00_2,13:00:00_3,13:00:00_4,13:00:00_5,13:00:00_6,13:00:00_7,13:15:00_0,13:15:00_1,13:15:00_2,13:15:00_3,13:15:00_4,13:15:00_5,13:15:00_6,13:15:00_7,13:30:00_0,13:30:00_1,13:30:00_2,13:30:00_3,13:30:00_4,13:30:00_5,13:30:00_6,13:30:00_7,13:45:00_0,13:45:00_1,13:45:00_2,13:45:00_3,13:45:00_4,13:45:00_5,13:45:00_6,13:45:00_7,14:00:00_0,14:00:00_1,14:00:00_2,14:00:00_3,14:00:00_4,14:00:00_5,14:00:00_6,14:00:00_7,14:15:00_0,14:15:00_1,14:15:00_2,14:15:00_3,14:15:00_4,14:15:00_5,14:15:00_6,14:15:00_7,14:30:00_0,14:30:00_1,14:30:00_2,14:30:00_3,14:30:00_4,14:30:00_5,14:30:00_6,14:30:00_7,14:45:00_0,14:45:00_1,14:45:00_2,14:45:00_3,14:45:00_4,14:45:00_5,14:45:00_6,14:45:00_7,15:00:00_0,15:00:00_1,15:00:00_2,15:00:00_3,15:00:00_4,15:00:00_5,15:00:00_6,15:00:00_7,15:15:00_0,15:15:00_1,15:15:00_2,15:15:00_3,15:15:00_4,15:15:00_5,15:15:00_6,15:15:00_7,15:30:00_0,15:30:00_1,15:30:00_2,15:30:00_3,15:30:00_4,15:30:00_5,15:30:00_6,15:30:00_7,15:45:00_0,15:45:00_1,15:45:00_2,15:45:00_3,15:45:00_4,15:45:00_5,15:45:00_6,15:45:00_7,16:00:00_0,16:00:00_1,16:00:00_2,16:00:00_3,16:00:00_4,16:00:00_5,16:00:00_6,16:00:00_7,16:15:00_0,16:15:00_1,16:15:00_2,16:15:00_3,16:15:00_4,16:15:00_5,16:15:00_6,16:15:00_7,16:30:00_0,16:30:00_1,16:30:00_2,16:30:00_3,16:30:00_4,16:30:00_5,16:30:00_6,16:30:00_7,16:45:00_0,16:45:00_1,16:45:00_2,16:45:00_3,16:45:00_4,16:45:00_5,16:45:00_6,16:45:00_7,17:00:00_0,17:00:00_1,17:00:00_2,17:00:00_3,17:00:00_4,17:00:00_5,17:00:00_6,17:00:00_7,17:15:00_0,17:15:00_1,17:15:00_2,17:15:00_3,17:15:00_4,17:15:00_5,17:15:00_6,17:15:00_7,17:30:00_0,17:30:00_1,17:30:00_2,17:30:00_3,17:30:00_4,17:30:00_5,17:30:00_6,17:30:00_7,17:45:00_0,17:45:00_1,17:45:00_2,17:45:00_3,17:45:00_4,17:45:00_5,17:45:00_6,17:45:00_7
0,Monday,Mission St 07th St - Julia St,0.072702,0.072702,0.072702,0.072702,0.072702,0.072702,0.072702,0.072702,0.037037,0.037037,0.037037,0.037037,0.037037,0.037037,0.037037,0.072702,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.037037,0.037037,0.000000,0.000000,0.000000,0.000000,0.000000,0.037037,0.037037,0.037037,0.000000,0.000000,0.000000,0.000000,0.037037,0.037037,0.037037,0.108368,0.000000,0.000000,0.000000,0.037037,0.037037,0.037037,0.108368,0.108368,0.000000,0.000000,0.037037,0.037037,0.037037,0.108368,0.108368,0.141391,0.000000,0.037037,0.037037,0.037037,0.108368,0.108368,0.141391,0.141391,0.037037,0.037037,0.037037,0.108368,0.108368,0.141391,0.141391,0.141391,0.037037,0.037037,0.108368,0.108368,0.141391,0.141391,0.141391,0.141391,0.000000,0.074074,0.074074,0.108368,0.108368,0.108368,0.108

In [13]:
# melting dataframe so there are 5 columns: weekday, street name, time, offset (0-7 for max 2 hours), probabiltiy of getting ticket

melted = pd.melt(offset_with_streets, id_vars=['days', 'streets'], var_name='time_and_number', value_name='prob')
melted

# plit time_and_number column into separate time and number columns
melted[['time', 'offset']] = melted['time_and_number'].str.split('_', expand=True)

melted.drop("time_and_number", axis = 1, inplace=True)
offset_pivoted = melted[["days", "streets", "time", "offset", "prob"]]


In [14]:
offset_pivoted

,days,streets,time,offset,prob
0,Monday,Mission St 07th St - Julia St,09:00:00,0,0.072702
1,Tuesday,Mission St 07th St - Julia St,09:00:00,0,0.072702
2,Wednesday,Mission St 07th St - Julia St,09:00:00,0,0.037037
3,Thursday,Mission St 07th St - Julia St,09:00:00,0,0.108368
4,Friday,Mission St 07th St - Julia St,09:00:00,0,0.074074
...,...,...,...,...,...
3532027,Wednesday,Park St Start: 01-95 Block - Mission St,17:45:00,7,0.000000
3532028,Thursday,Park St Start: 01-95 Block - Mission St,17:45:00,7,0.000000
3532029,Friday,Park St Start: 01-95 Block - Mission St,17:45:00,7,0.000000
3532030,Saturday,Park St Start: 01-95 Block - Mission St,17:45:00,7,0.000000


In [15]:
offset_pivoted.to_csv("../data/probs_with_offset.csv", index = False)